<a href="https://colab.research.google.com/github/Nskoushik26276/Fake-News-Detection-with-Multimodal-Cues-Text-Image-/blob/main/Real_or_Fake_news_detection_using_multimodal_frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MOUNTING DRIVE**

In [7]:
#drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#content viewing
!ls "/content/drive/MyDrive"

'Adobe - LeetCode.gdoc'		       'EML WEEK-6 - Jupyter Notebook.pdf'
'Colab Notebooks'		        fakenewsdetection.zip
'DocScanner Feb 8, 2025 7-49 PM.gdoc'   spring.demo1
'DocScanner Feb 8, 2025 7-49 PM.pdf'   'weather-monitoring.zip(AI-A-44).zip'
'EML WEEK-6 - Jupyter Notebook.gdoc'


In [9]:
#path conformation
!ls "/content/drive/MyDrive/fakenewsdetection.zip"

/content/drive/MyDrive/fakenewsdetection.zip


**Unzip Dataset**

In [10]:
import zipfile
import os

# Path to your zip file
zip_path = "/content/drive/MyDrive/fakenewsdetection.zip"
extract_path = "/content/fakenewsdata"

# Extract if not already
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
    print("✅ Dataset extracted successfully!")
else:
    print("⚡ Already extracted")

# Count files and folders
file_count = 0
folder_count = 0
for root, dirs, files in os.walk(extract_path):
    folder_count += len(dirs)
    file_count += len(files)

print(f"📂 Total folders: {folder_count}")
print(f"📄 Total files: {file_count}")

# Show a few sample files
print("\n🔎 Sample contents:")
for i, (root, dirs, files) in enumerate(os.walk(extract_path)):
    print("📁", root)
    for f in files[:5]:  # show first 5 files only
        print("   └──", f)
    if i > 2:  # limit to top 3 levels
        break

✅ Dataset extracted successfully!
📂 Total folders: 3
📄 Total files: 6

🔎 Sample contents:
📁 /content/fakenewsdata
📁 /content/fakenewsdata/Fakeddit datasetv2.0
📁 /content/fakenewsdata/Fakeddit datasetv2.0/multimodal_only_samples
   └── multimodal_train.tsv
   └── multimodal_test_public.tsv
   └── multimodal_validate.tsv
📁 /content/fakenewsdata/Fakeddit datasetv2.0/all_samples (also includes non multimodal)
   └── all_validate.tsv
   └── all_train.tsv
   └── all_test_public.tsv


**Explore Dataset Files**

In [11]:
# List top-level contents
os.listdir(extract_path)

['Fakeddit datasetv2.0']

In [12]:
import os
for root, dirs, files in os.walk(extract_path):
    print("📂", root)
    for f in files[:10]:  # show first 10 files in each folder
        print("   └──", f)
    print("-" * 50)

📂 /content/fakenewsdata
--------------------------------------------------
📂 /content/fakenewsdata/Fakeddit datasetv2.0
--------------------------------------------------
📂 /content/fakenewsdata/Fakeddit datasetv2.0/multimodal_only_samples
   └── multimodal_train.tsv
   └── multimodal_test_public.tsv
   └── multimodal_validate.tsv
--------------------------------------------------
📂 /content/fakenewsdata/Fakeddit datasetv2.0/all_samples (also includes non multimodal)
   └── all_validate.tsv
   └── all_train.tsv
   └── all_test_public.tsv
--------------------------------------------------


**Load Text + Metadata**

In [13]:
import pandas as pd
import os
base_path = "/content/fakenewsdata/Fakeddit datasetv2.0/multimodal_only_samples"
# Load splits
train_df = pd.read_csv(os.path.join(base_path, "multimodal_train.tsv"), sep="\t")
val_df   = pd.read_csv(os.path.join(base_path, "multimodal_validate.tsv"), sep="\t")
test_df  = pd.read_csv(os.path.join(base_path, "multimodal_test_public.tsv"), sep="\t")
print("✅ Data loaded successfully!")
print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)
# Quick preview
print("\n🔎 Columns:", train_df.columns.tolist())
print(train_df.head())

✅ Data loaded successfully!
Train shape: (564000, 16)
Validation shape: (59342, 16)
Test shape: (59319, 16)

🔎 Columns: ['author', 'clean_title', 'created_utc', 'domain', 'hasImage', 'id', 'image_url', 'linked_submission_id', 'num_comments', 'score', 'subreddit', 'title', 'upvote_ratio', '2_way_label', '3_way_label', '6_way_label']
           author                                        clean_title  \
0     Alexithymia  my walgreens offbrand mucinex was engraved wit...   
1        VIDCAs17                this concerned sink with a tiny hat   
2  prometheus1123      hackers leak emails from uae ambassador to us   
3             NaN                           puppy taking in the view   
4       3rikR3ith               i found a face in my sheet music too   

    created_utc         domain  hasImage      id  \
0  1.551641e+09    i.imgur.com      True  awxhir   
1  1.534727e+09      i.redd.it      True  98pbid   
2  1.496511e+09  aljazeera.com      True  6f2cy5   
3  1.471341e+09    i.imgu

**(Optional) Download Images**

In [14]:
#image download
import requests
from tqdm import tqdm
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png",
    "https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg"
]
# More realistic headers to avoid 403 errors
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://upload.wikimedia.org/"
}

for url in tqdm(image_urls):
    try:
        response = requests.get(url, headers=headers)
        filename = url.split("/")[-1]
        if response.status_code == 200:
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"✅ Successfully downloaded: {filename}")
        else:
            print(f"⚠️ Failed to download: {url} (Status code: {response.status_code})")
    except Exception as e:
        print(f"⚠️ Error downloading {url}: {e}")
print("All downloads attempted!")

100%|██████████| 2/2 [00:00<00:00, 63.90it/s]

✅ Successfully downloaded: PNG_transparency_demonstration_1.png
✅ Successfully downloaded: Black_square.jpg
All downloads attempted!


**Prepare Data for Modeling**

In [15]:
#text encoding
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
text = "Breaking news: something happened!"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
text_embedding = outputs.last_hidden_state.mean(dim=1)  # Pooled embedding

In [16]:
#Image Encoding
import torch
from torchvision import models, transforms
from PIL import Image
# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Image filename
filename = "PNG_transparency_demonstration_1.png"
try:
    # Open and convert image to RGB
    image = Image.open(filename).convert("RGB")
    # Preprocess and add batch dimension
    image_tensor = preprocess(image).unsqueeze(0)  # Shape: [1, 3, 224, 224]
    print(f"{filename} loaded successfully, tensor shape: {image_tensor.shape}")
    # Load pretrained ResNet50
    resnet = models.resnet50(pretrained=True)
    # Remove the final classifier layer
    resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
    resnet.eval()  # Set to eval mode
    # Get image embedding
    with torch.no_grad():
        image_embedding = resnet(image_tensor).flatten(1)  # Shape: [1, 2048]
    print(f"Image embedding shape: {image_embedding.shape}")
except FileNotFoundError:
    print(f"❌ File not found: {filename}")
except Exception as e:
    print(f"⚠️ Error processing {filename}: {e}")

PNG_transparency_demonstration_1.png loaded successfully, tensor shape: torch.Size([1, 3, 224, 224])


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Image embedding shape: torch.Size([1, 2048])


In [17]:
#Fusion of Text + Image Embeddings
import torch
import torch.nn as nn
# Dummy embeddings (replace with actual BERT & ResNet embeddings)
text_embedding = torch.randn(1, 768)   # Example: BERT/DistilBERT output
image_embedding = torch.randn(1, 2048) # Example: ResNet output
# Step 1: Fuse embeddings
fused_embedding = torch.cat([text_embedding, image_embedding], dim=1)
print("Fused embedding shape:", fused_embedding.shape)  # Should be [1, 2816]
# Step 2: Define a simple classifier
classifier = nn.Sequential(
    nn.Linear(fused_embedding.size(1), 512),
    nn.ReLU(),
    nn.Linear(512, 2),   # Example: 2 classes (real/fake)
    nn.Softmax(dim=1)
)
# Step 3: Pass fused embedding to classifier
output = classifier(fused_embedding)
print("Classifier output:", output)
print("Predicted class:", torch.argmax(output, dim=1).item())

Fused embedding shape: torch.Size([1, 2816])
Classifier output: tensor([[0.4412, 0.5588]], grad_fn=<SoftmaxBackward0>)
Predicted class: 1


**Prepare Data for Modeling and output together**

In [19]:
!pip install newspaper3k
!pip install lxml_html_clean
# ---------------- 0. Install necessary packages ----------------
!pip install -q transformers torch torchvision beautifulsoup4 lxml_html_clean

# ---------------- 1. Imports ----------------
import torch
from torch import nn, optim
from torchvision import models, transforms
from PIL import Image
from transformers import DistilBertTokenizer, DistilBertModel
from bs4 import BeautifulSoup
import requests
from google.colab import files

# ---------------- 2. Text Encoder ----------------
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
text_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = text_model(**inputs)
    cls_embedding = outputs.last_hidden_state[:,0,:]
    return cls_embedding

# ---------------- 3. Image Encoder ----------------
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove classifier
resnet.eval()

def encode_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"❌ File not found: {image_path}")
        return None
    image_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        embedding = resnet(image_tensor).flatten(1)
    return embedding

# ---------------- 4. Multimodal Classifier ----------------
class MultimodalClassifier(nn.Module):
    def __init__(self, text_dim=768, image_dim=2048, hidden_dim=512, num_classes=2):
        super().__init__()
        self.fc1 = nn.Linear(text_dim + image_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, text_emb, image_emb):
        if image_emb is None:
            image_emb = torch.zeros((text_emb.size(0), 2048))
        fused = torch.cat([text_emb, image_emb], dim=1)
        x = self.fc1(fused)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

classifier = MultimodalClassifier()

# ---------------- 5. Training Setup ----------------
# Example dummy dataset (replace with real labeled dataset)
train_texts = [
    "Indian Railways adds more coaches to Vande Bharat trains",
    "Aliens discovered on Mars"
]
train_image_paths = [
    None,  # No image for first example
    None   # No image for second example
]
train_labels = [0, 1]  # 0 = Real, 1 = Fake

optimizer = optim.Adam(classifier.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ---------------- 6. Training Loop ----------------
classifier.train()
for epoch in range(5):
    total_loss = 0
    for text, img_path, label in zip(train_texts, train_image_paths, train_labels):
        text_emb = encode_text(text)
        img_emb = encode_image(img_path) if img_path else None
        label_tensor = torch.tensor([label])

        optimizer.zero_grad()
        output = classifier(text_emb, img_emb)
        loss = criterion(output, label_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Save the trained model
torch.save(classifier.state_dict(), "trained_classifier.pth")
print("✅ Trained classifier saved!")

# ---------------- 7. User Input ----------------
def fetch_news_text(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    try:
        resp = requests.get(url, headers=headers)
        if resp.status_code != 200:
            print(f"❌ Failed to fetch URL: {resp.status_code}")
            return None
        soup = BeautifulSoup(resp.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = " ".join([p.get_text() for p in paragraphs])
        return text if text.strip() else None
    except Exception as e:
        print(f"❌ Error fetching URL: {e}")
        return None

print("\nChoose input method:")
print("1 - Enter text and upload image")
print("2 - Enter news URL")
choice = input("Enter 1 or 2: ").strip()

text_input = None
image_embedding = None

if choice == "1":
    text_input = input("Enter a description text: ")
    print("Upload an image file from your system:")
    uploaded = files.upload()
    if uploaded:
        image_path_input = list(uploaded.keys())[0]
        image_embedding = encode_image(image_path_input)
elif choice == "2":
    url_input = input("Enter the news URL: ").strip()
    text_input = fetch_news_text(url_input)
    if text_input is None:
        text_input = input("Enter news text manually (URL scraping failed): ")
    print("Upload an image file (optional, press Enter to skip):")
    uploaded = files.upload()
    if uploaded:
        image_path_input = list(uploaded.keys())[0]
        image_embedding = encode_image(image_path_input)
else:
    print("❌ Invalid choice. Exiting.")
    exit()

# ---------------- 8. Load trained classifier ----------------
classifier.load_state_dict(torch.load("trained_classifier.pth"))
classifier.eval()

# ---------------- 9. Encode Text & Predict ----------------
text_embedding = encode_text(text_input)
output = classifier(text_embedding, image_embedding)

# ---------------- 10. Show Result ----------------
class_names = ["Real News", "Fake News"]
if output is not None:
    prob_real, prob_fake = output[0].tolist()
    print(f"\nClassifier output (probabilities): Real: {prob_real:.3f}, Fake: {prob_fake:.3f}")
    print("Predicted class:", class_names[torch.argmax(output, dim=1).item()])
else:
    print("Prediction could not be made.")

Epoch 1, Loss: 1.4263
Epoch 2, Loss: 1.3591
Epoch 3, Loss: 1.3123
Epoch 4, Loss: 1.2722
Epoch 5, Loss: 1.2350
✅ Trained classifier saved!

Choose input method:
1 - Enter text and upload image
2 - Enter news URL
Enter 1 or 2: 2
Enter the news URL: https://economictimes.indiatimes.com/industry/transportation/railways/railways-to-run-longer-vande-bharat-trains-as-demand-rises/articleshow/123566653.cms?from=mdr
Upload an image file (optional, press Enter to skip):



Classifier output (probabilities): Real: 0.630, Fake: 0.370
Predicted class: Real News
